Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this assignment to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>

Caution: Since this is real language on Twitter and deals with world events, some of it could be disturbing. In the later section of the course, we will deal with ethics and what is appropriate and what is not. 

First, let's read the tweets from the Google drive and print a few lines from the file.



Run the below code snippet. It will generate a URL which generates an authorization code. Enter it below to give Colab access to your Google drive. 

**Note:** Copy button may not work. Try copying the authorization code manually.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


When you run the `ls` command below, you should see the files in the tweets folder.




In [2]:
!ls "/content/drive/My Drive/tweets"

20000_tweets.jsonl
20000_tweets.txt
covid-tweets-2020-08-10-2020-08-21.tokenized.txt
covid-tweets-2020-08-10-2020-08-21.trigrams.txt
GoogleNews-vectors-negative300.bin.gz
stop_words.txt


Let's read the top 10 tweets from the file `20000_tweets.txt` and print them.

In [3]:
f = open("/content/drive/My Drive/tweets/20000_tweets.txt", "r")

line_count = 0
top_tweets = []
for tweet in f:
  print("### Tweet", line_count, "#####")
  print(tweet)
  
  top_tweets.append(tweet)
  line_count += 1
  if line_count >= 10:
    break
f.close()

### Tweet 0 #####
Covid-19 Economic Response: Cancel Student Loans by Executive Order. - Sign the Petition! https://t.co/BnPXWHv5cr via @Change

### Tweet 1 #####
Hey! The stock markets up! Fuck Trump and the Trumpublican Senate! https://t.co/4t6mgbaG2C

### Tweet 2 #####
@ProjectLincoln blame Trump for no sports because of Covid19 @MeidasTouch

### Tweet 3 #####
Do u guys think that reopening of the dinning in the restaurants of Lahore will bring any change in our lives? Nothing is going to change I think cause many people have learned cooking, painting and many new stuff during lockdown and I think we all will continue those things 1/2

### Tweet 4 #####
@NBCSAthletics Ya just knew the season wouldn’t go by without some bench clearing brawls... Covid or not. Behaviors can’t be changed because of rules, sadly

### Tweet 5 #####
You’re comparing apples and oranges. The NCAA has a lot more to balance than the NFL. https://t.co/oeMsbgeem7

### Tweet 6 #####
THANK YOU @POTUS Absolutely NO

# **Problem 1: Word Tokenizer**

Implement your own tokenizer for tweets without using libraries like `nltk`, `spacy` or any existing tokenizers. Tokenizer is a fundamental tool for processing any text data. The tokenizer should preserve punctuations.

The outputs for the top five tweets should approximately be as follows (it's fine to not exactly match this output)

```
['Covid-19', 'Economic', 'Response', ':', 'Cancel', 'Student', 'Loans', 'by', 'Executive', 'Order', '.', '-', 'Sign', 'the', 'Petition', '!', 'https://t.co/BnPXWHv5cr', 'via', '@Change']
['Hey', '!', 'The', 'stock', 'markets', 'up', '!', 'Fuck', 'Trump', 'and', 'the', 'Trumpublican', 'Senate', '!', 'https://t.co/4t6mgbaG2C']
['@ProjectLincoln', 'blame', 'Trump', 'for', 'no', 'sports', 'because', 'of', 'Covid19', '@MeidasTouch']
['Do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'Lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', '?', 'Nothing', 'is', 'going', 'to', 'change', 'I', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', ',', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'I', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@NBCSAthletics', 'Ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', '...', 'Covid', 'or', 'not', '.', 'Behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', ',', 'sadly']
```

A thing to note is that words like `wouldn't` and `you'll` become two tokens `would n't` and `you 'll`, after tokenization. Treating `'nt` and  `'ll` as independent tokens is common in NLP tools. You can come up with that kind of list easily, e.g., `'s, 've`.


In [4]:
# import any relevant libraries here
import string
import re
import copy

common_contractions_2 = ["'d", "'m", "'s", "’d", "’m", "’s"]
common_contractions_3 = ["'ll", "n't", "'re", "'ve", "’ll", "n’t", "’re", "’ve"]

# special contractions
sc_list = ["cannot", "gimme", "gonna", "gotta", "'tis", "’tis", "'twas", "’twas", "wanna", "y'all", "y’all"]
sc_dic = {"cannot": ["can", "not"],
          "gimme": ["gim", "me"],
          "gonna": ["gon", "na"],
          "gotta": ["got", "ta"],
          "'tis": ["'t", "is"],
          "’tis": ["’t", "is"],
          "'twas": ["'t", "was"],
          "’twas": ["’t", "was"],
          "wanna": ["wan", "na"],
          "y'all": ["y", "'all"],
          "y’all": ["y", "’all"]
          }

# a function to tokenize text into words
def tokenize(text):
    # split text
    # website OR number with .,/ OR hashtag OR username OR word with -'’ characters OR punctuation
    words = re.findall(r"https:[^\s]+|[\d][\d\.,/]+[\d]+|[#][\w]+|[@][\w]+|[\w\-'’]+|[^\s\w\-'’]", text)

    words_c = copy.deepcopy(words)

    # handle contractions
    for word in words:
        # common contractions
        # check if there is a ' or ’
        apostrophe_1 = "'"
        apostrophe_2 = "’"

        if (apostrophe_1 in word) or (apostrophe_2 in word):

            if (len(word) > 3) and (word[-3:] in common_contractions_3):
                # split word
                word_1 = word[:-3]
                word_2 = word[-3:]

                # replace contraction with two words
                i = words_c.index(word)
                words_c[i:i+1] = word_1, word_2

            elif (len(word) > 2) and (word[-2:] in common_contractions_2):
                # split word
                word_1 = word[:-2]
                word_2 = word[-2:]

                # replace contraction with two words
                i = words_c.index(word)
                words_c[i:i+1] = word_1, word_2

        # special contractions
        if word in sc_list:
            # split word
            word_1 = sc_dic[word][0]
            word_2 = sc_dic[word][1]

            # replace contraction with two words
            i = words_c.index(word)
            words_c[i:i+1] = word_1, word_2

    return words_c

tokenized_top_tweets = [tokenize(tweet) for tweet in top_tweets]

for tokenized_tweet in tokenized_top_tweets:
    print(tokenized_tweet)

['Covid-19', 'Economic', 'Response', ':', 'Cancel', 'Student', 'Loans', 'by', 'Executive', 'Order', '.', '-', 'Sign', 'the', 'Petition', '!', 'https://t.co/BnPXWHv5cr', 'via', '@Change']
['Hey', '!', 'The', 'stock', 'markets', 'up', '!', 'Fuck', 'Trump', 'and', 'the', 'Trumpublican', 'Senate', '!', 'https://t.co/4t6mgbaG2C']
['@ProjectLincoln', 'blame', 'Trump', 'for', 'no', 'sports', 'because', 'of', 'Covid19', '@MeidasTouch']
['Do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'Lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', '?', 'Nothing', 'is', 'going', 'to', 'change', 'I', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', ',', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'I', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@NBCSAthletics', 'Ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 

```
['Covid-19', 'Economic', 'Response', ':', 'Cancel', 'Student', 'Loans', 'by', 'Executive', 'Order', '.', '-', 'Sign', 'the', 'Petition', '!', 'https://t.co/BnPXWHv5cr', 'via', '@Change']
['Hey', '!', 'The', 'stock', 'markets', 'up', '!', 'Fuck', 'Trump', 'and', 'the', 'Trumpublican', 'Senate', '!', 'https://t.co/4t6mgbaG2C']
['@ProjectLincoln', 'blame', 'Trump', 'for', 'no', 'sports', 'because', 'of', 'Covid19', '@MeidasTouch']
['Do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'Lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', '?', 'Nothing', 'is', 'going', 'to', 'change', 'I', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', ',', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'I', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@NBCSAthletics', 'Ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', '...', 'Covid', 'or', 'not', '.', 'Behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', ',', 'sadly']
```

In [5]:
# compare output to nltk
# import nltk
# nltk.download('punkt')

# tokenized_top_tweets = [nltk.word_tokenize(tweet) for tweet in top_tweets]

# for tokenized_tweet in tokenized_top_tweets:
    # print(tokenized_tweet)

In [6]:
# test code
# import nltk
# nltk.download('punkt')

# s = "hello world"
# print(tokenize(s))
# print(nltk.word_tokenize(s))

# Problem 2: Post-process the tokenized tweets

Clean the tokenized tweets such that usernames are repalced with `@USER`, URLs are replaced with `URL`, punctuations are removed and words are lower-cased.

The output for the top ten tweets after cleaning should look something like this

```
['covid-19', 'economic', 'response', 'cancel', 'student', 'loans', 'by', 'executive', 'order', 'sign', 'the', 'petition', 'URL', 'via', '@USER']
['hey', 'the', 'stock', 'markets', 'up', 'fuck', 'trump', 'and', 'the', 'trumpublican', 'senate', 'URL']
['@USER', 'blame', 'trump', 'for', 'no', 'sports', 'because', 'of', 'covid19', '@USER']
['do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', 'nothing', 'is', 'going', 'to', 'change', 'i', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'i', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@USER', 'ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', 'covid', 'or', 'not', 'behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', 'sadly']
```

In [7]:
punctuation = ["!", "\"", "#", "$", "%", "&", "'", "(", ")", "*", "+", ",", "-",
               ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~"]

# function to clean a tweet
def clean_a_tweet(tokenized_tweet):
    to_remove = []

    clean_tokenized_tweet = copy.deepcopy(tokenized_tweet)

    # remove punctuation
    for word in clean_tokenized_tweet:
        if word in punctuation:
            to_remove.append(word)
    
    for r in to_remove:
        clean_tokenized_tweet.remove(r)

    for word in clean_tokenized_tweet:
        # replace usernames with @USER
        if (len(word) > 1) and (word[0] == "@"):
            i = clean_tokenized_tweet.index(word)
            clean_tokenized_tweet[i] = "@USER"

        # replace URLs with URL
        elif (len(word) > 6) and (word[:6] == "https:"):
            i = clean_tokenized_tweet.index(word)
            clean_tokenized_tweet[i] = "URL"

        # make words lower-case
        else:
            word_lc = word.lower()
            
            i = clean_tokenized_tweet.index(word)
            clean_tokenized_tweet[i] = word_lc

    return clean_tokenized_tweet

anonymized_top_tweets = [clean_a_tweet(tokenized_tweet) for tokenized_tweet in tokenized_top_tweets]
for tokenized_tweet in anonymized_top_tweets:
    print(tokenized_tweet)

['covid-19', 'economic', 'response', 'cancel', 'student', 'loans', 'by', 'executive', 'order', 'sign', 'the', 'petition', 'URL', 'via', '@USER']
['hey', 'the', 'stock', 'markets', 'up', 'fuck', 'trump', 'and', 'the', 'trumpublican', 'senate', 'URL']
['@USER', 'blame', 'trump', 'for', 'no', 'sports', 'because', 'of', 'covid19', '@USER']
['do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', 'nothing', 'is', 'going', 'to', 'change', 'i', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'i', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@USER', 'ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', 'covid', 'or', 'not', 'behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', 'sadly']
['y

```
['covid-19', 'economic', 'response', 'cancel', 'student', 'loans', 'by', 'executive', 'order', 'sign', 'the', 'petition', 'URL', 'via', '@USER']
['hey', 'the', 'stock', 'markets', 'up', 'fuck', 'trump', 'and', 'the', 'trumpublican', 'senate', 'URL']
['@USER', 'blame', 'trump', 'for', 'no', 'sports', 'because', 'of', 'covid19', '@USER']
['do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', 'nothing', 'is', 'going', 'to', 'change', 'i', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'i', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@USER', 'ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', 'covid', 'or', 'not', 'behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', 'sadly']
```

In [8]:
# test code
# tester = ["hello", "Hello", "HELLO"]
# clean_a_tweet(tester)

# Problem 3: Unigrams and Bigrams

Read **all** tweets from the file `20000_tweets.txt` and print the top 10 unigrams and bigrams. Consider only the English tweets. You have to use `tokenize` and `clean_a_tweet` function.

Output format

```
Top 10 Unigrams
@USER	16698
the	16666
URL	12610
to	11866
a	9845
....
....
....
....
....

Top 10 Bigrams
@USER @USER	7494
in the	1280
of the	1265
it ’s	860
a mask	793
....
....
....
....
....
```

Your numbers need not match the above word frequencies. Depending on your tokenizer, you may get different numbers

In [9]:
# Write your code to build top 10 unigrams and bigrams
# Hint: You can use python dictionary or collections.Counter().
from collections import Counter

# read all tweets from 20000_tweets.txt
f = open("/content/drive/My Drive/tweets/20000_tweets.txt", "r")

all_tweets = []
for tweet in f:
  all_tweets.append(tweet)

f.close()

# tokenize all tweets
tokenized_all_tweets = [tokenize(tweet) for tweet in all_tweets]

# clean all tweets
anonymized_all_tweets = [clean_a_tweet(tokenized_tweet) for tokenized_tweet in tokenized_all_tweets]

# get unigrams
def get_unigrams(list_of_tweets, top_n=10, silent=False):
    # one list for all unigrams
    all_unigrams = []
    for tokenized_tweet in list_of_tweets:
        all_unigrams = all_unigrams + tokenized_tweet

    unigram_count = Counter(all_unigrams)

    if not silent:
    # print the top n unigrams
        top_n_unigrams = unigram_count.most_common(top_n)
        print('Top', top_n, 'Unigrams')
        for unigram in top_n_unigrams:
            print(unigram[0], '    ', unigram[1])

    return dict(unigram_count)

unigrams = get_unigrams(anonymized_all_tweets)
print()

# get bigrams
def get_bigrams(list_of_tweets, top_n=10, silent=False):
    # one list for all bigrams
    all_bigrams = []
    for tokenized_tweet in list_of_tweets:
        if len(tokenized_tweet) > 1:
            for i in range(len(tokenized_tweet)-1):
                bigram = tokenized_tweet[i] + " " + tokenized_tweet[i+1]
                all_bigrams.append(bigram)

    bigram_count = Counter(all_bigrams)

    if not silent:
    # print the top n bigrams
        top_n_bigrams = bigram_count.most_common(top_n)
        print('Top', top_n, 'Bigrams')
        for bigram in top_n_bigrams:
            print(bigram[0], '    ', bigram[1])

    return dict(bigram_count)

bigrams = get_bigrams(anonymized_all_tweets)

Top 10 Unigrams
@USER      16837
the      16662
URL      12624
to      11855
a      9851
and      8544
of      7810
i      6576
is      6475
in      6301

Top 10 Bigrams
@USER @USER      7532
in the      1280
of the      1264
it ’s      856
a mask      790
this is      748
@USER i      725
the pandemic      717
wear a      698
URL URL      692


```
Top 10 Unigrams
@USER	16698
the	16666
URL	12610
to	11866
a	9845
...

Top 10 Bigrams
@USER @USER	7494
in the	1280
of the	1265
it ’s	860
a mask	793
...
```

In [10]:
# test code
# list(unigrams)[-25:]

# Problem 4: How frequent are the following unigrams and bigrams?

```
covid
coronavirus
republicans
democrats
social distancing
wear mask
no mask
donald trump
joe biden
```

Output:
```
covid	4437
coronavirus	1368
...
...
social distancing	568
wear mask	19
...
...
...
```


In [11]:
# Write your code to print the frequencies of the above unigrams and bigrams
# Hint: reuse the dictionaries you built in the previous cell
word_list_1 = ["covid", "coronavirus", "republicans", "democrats"]
word_list_2 = ["social distancing", "wear mask", "no mask", "donald trump", "joe biden"]

for word in word_list_1:
  if word in unigrams:
    print(f"{word}    {unigrams[word]}")

for word in word_list_2:
  if word in bigrams:
    print(f"{word}    {bigrams[word]}")

covid    4344
coronavirus    1353
republicans    58
democrats    160
social distancing    558
wear mask    19
no mask    19
donald trump    62
joe biden    23
